In [38]:
import os

from dotenv import dotenv_values
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

env_vars = dotenv_values('.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

llm = ChatOpenAI(temperature=0.1)

# 대화를 요약해서 저장
memory = ConversationSummaryMemory(
    llm=llm
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi, I'm moztiq, I live in South Korea", "Wow that is so cool!")


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hi, I'm moztiq, I live in South Korea\nAI: Wow that is so cool!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman:

In [39]:
add_message("South Korea is so pretty", "I wish I could go!")
get_history()

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "The human introduces themselves as moztiq and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool.",
  "new_lines": "Human: South Korea is so pretty\nAI: I wish I could go!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good

{'history': 'The human introduces themselves as moztiq and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool. The human mentions that South Korea is pretty, to which the AI responds by expressing a desire to go there.'}